In [16]:
from gensim.summarization import summarize
import fse # fast sentence embeddings

from gensim.models import KeyedVectors

path_w2v = '/home/santosh/Work/models/word2vec/CORD-19/CORD-10-w2v_200d_5w_10i_3mc.bin'

covid_trained_model = KeyedVectors.load_word2vec_format(path_w2v, binary=True)

2020-04-04 19:28:34,422 : INFO : loading projection weights from /home/santosh/Work/models/word2vec/CORD-19/CORD-10-w2v_200d_5w_10i_3mc.bin
2020-04-04 19:28:39,524 : INFO : loaded (241336, 200) matrix from /home/santosh/Work/models/word2vec/CORD-19/CORD-10-w2v_200d_5w_10i_3mc.bin


In [1]:
# Here I am using word embeddings from the CORD-19
import glob
import nltk
import logging
import os
from tqdm import tqdm

# start the log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

# get the current working directory and file
result_dir_path = '/home/santosh/Work/Datasets/CORD-19-paragraphs/'

root_path = '/home/santosh/Work/Datasets/CORD-19-research-challenge/'
paths = ['biorxiv_medrxiv/biorxiv_medrxiv/',
        'comm_use_subset/comm_use_subset/',
        'noncomm_use_subset/noncomm_use_subset/']




In [2]:
# get all the jsosn
import json

all_jsons =[]
for json_path in paths:
     all_jsons.extend(sorted(glob.glob(root_path+json_path + '*.json*')))


In [3]:
all_jsons[0]

'/home/santosh/Work/Datasets/CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/0015023cc06b5362d332b3baf348d11567ca2fbb.json'

In [4]:
import pandas as pd


meta_df = pd.read_csv('/home/santosh/Work/Datasets/CORD-19-research-challenge/metadata.csv')
meta_df.head(1)

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,custom_license


In [5]:
selected_columns = ['sha', 'title', 'doi', 'pmcid', 'pubmed_id', 'publish_time', 'journal', 'authors'] # sha is paper id here

meta_ref_df = meta_df[selected_columns]
meta_ref_df = meta_ref_df[meta_ref_df['sha'].notna()]

In [6]:
meta_ref_df.head(10)

,sha,title,doi,pmcid,pubmed_id,publish_time,journal,authors
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077.0,1973-08-31,The American Journal of Medicine,"Brunner, Carolyn M.; Horwitz, David A.; Shann,..."
5,212e990b378e8d267042753d5f9d4a64ea5e9869,Infectious diarrhea: Pathogenesis and risk fac...,10.1016/0002-9343(85)90367-5,NaN,2861742.0,1985-06-28,The American Journal of Medicine,"Cantey, J.Robert"
6,bf5d344243153d58be692ceb26f52c08e2bd2d2f,New perspectives on the pathogenesis of rheuma...,10.1016/0002-9343(88)90356-7,NaN,3052052.0,1988-10-14,The American Journal of Medicine,"Zvaifler, Nathan J."
7,ddd2ecf42ec86ad66072962081e1ce4594431f9c,Management of acute and chronic respiratory tr...,10.1016/0002-9343(88)90456-1,NaN,3048091.0,1988-09-16,The American Journal of Medicine,"Ellner, Jerrold J."
8,a55cb4e724091ced46b5e55b982a14525eea1c7e,Acute bronchitis: Results of U.S. and European...,10.1016/0002-9343(92)90608-E,NaN,1621745.0,1992-06-22,The American Journal of Medicine,"Dere, Willard H."
9,a1fd28115cfa14869f29f30d2121c1058e6e81e9,Clinical and Immunologic Responses in Patients...,10.1016/0002-9394(75)90398-0,NaN,170831.0,1975-10-31,American Journal of Ophthalmology,"Knopf, Harry L.S.; Hierholzer, John C."
10,60bf634cf15582a6b7f72eb3811ca5d8b8825c44,Determination of microsomal lauric acid hydrox...,10.1016/0003-2697(88)90093-0,NaN,3389520.0,1988-04-30,Analytical Biochemistry,"Romano, Maria C.; Straub, Kenneth M.; Yodis, L..."
11,b84658904af7f828615c7de04ffcb088bd49c50d,Phospholipid vesicles containing bovine heart ...,10.1016/0003-9861(91)90605-I,NaN,1716878.0,1991-10-31,Archives of Biochemistry and Biophysics,"Prochaska, Lawrence J.; Wilson, Kathryn S."
12,c05ffd04482059ba3cd3965a389003e814ee2699,The oligomeric structure of renal aminopeptida...,10.1016/0005-2736(93)90386-E,NaN,8093665.0,1993-01-18,Biochimica et Biophysica Acta (BBA) - Biomembr...,"Plakidou-Dymock, Stella; McGivan, John D."
14,0fa2750b5605f80e81064c6442a823ca3bf65f85,"Predict7, a program for protein structure pred...",10.1016/0006-291X(89)90049-1,NaN,2539121.0,1989-03-15,Biochemical and Biophysical Research Communica...,"Cármenes, R.S.; Freije, J.P.; Molina, M.M.; Ma..."


In [7]:
# extract sentences with doi
from dateutil import parser

count = 0
cord19_json_list = []

for each_json_file in tqdm(all_jsons):
   
    with open(each_json_file) as json_file:
        data = json.load(json_file)
        try:
            date_t = meta_ref_df[meta_ref_df['sha'].str.contains(data['paper_id'])]['publish_time'].values.tolist()[0]
                
            if parser.parse(date_t).year>2017:
                data_dict = {}
                
                try:
                    data_dict['source'] = meta_ref_df[meta_ref_df['sha'].str.contains(data['paper_id'])]['doi'].values.tolist()[0]
                except:
                    data_dict['source'] =''
                
                try:
                    data_dict['title'] = meta_ref_df[meta_ref_df['sha'].str.contains(data['paper_id'])]['title'].values.tolist()[0]
                except:
                    data_dict['title'] =''
                    
                try:
                    data_dict['authors'] = meta_ref_df[meta_ref_df['sha'].str.contains(data['paper_id'])]['authors'].values.tolist()[0]
                except:
                    data_dict['authors'] =''
                    
                
                
                full_text = []

                try:
                    for each_text in data['metadata']:
                        full_text.append(each_text['title'])
                except:
                        pass 


                try:
                    for each_text in data['abstract']:
                        full_text.append(each_text['text'])
                except:
                        pass

                try:
                    for each_text in data['body_text']:
                        full_text.append(each_text['text'])
                except:
                        pass

                try:
                    for each_text in data['ref_entries']:
                        full_text.append(data['ref_entries'][each_text]['text'])
                except:
                        pass    

                for each_para in full_text:
                    if each_para and 'doi:' not in each_para and 'word count' not in each_para and 'All rights reserved' not in each_para and 'No reuse allowed without permission' not in each_para:
                        data_dict['index_key']=count
                        data_dict['paragraph']=each_para
                        count = count+1
#                         print(count)
                        with open('/home/santosh/Work/Datasets/cord-19-json/'+'data_dict_paragraphs.json','a') as f:
                            json.dump(data_dict,f)
                            f.write('\n')
                        
    
        except:
            pass
                            

100%|██████████| 12356/12356 [06:05<00:00, 33.80it/s]


In [30]:
import nltk

paragraphs = []
with open('/home/santosh/Work/Datasets/cord-19-json/data_dict_paragraphs.json', 'r') as infile:
    for line in infile:
        data = json.loads(line)
        paragraphs.append(nltk.word_tokenize(data['paragraph']))


In [31]:
paragraphs[0]

['VP3',
 ',',
 'and',
 'VP0',
 '(',
 'which',
 'is',
 'further',
 'processed',
 'to',
 'VP2',
 'and',
 'VP4',
 'during',
 'virus',
 'assembly',
 ')',
 '(',
 '6',
 ')',
 '.',
 'The',
 'P2',
 '64',
 'and',
 'P3',
 'regions',
 'encode',
 'the',
 'non-structural',
 'proteins',
 '2B',
 'and',
 '2C',
 'and',
 '3A',
 ',',
 '3B',
 '(',
 '1-3',
 ')',
 '(',
 'VPg',
 ')',
 ',',
 '3C',
 'pro',
 'and',
 '4',
 'structural',
 'protein-coding',
 'region',
 'is',
 'replaced',
 'by',
 'reporter',
 'genes',
 ',',
 'allow',
 'the',
 'study',
 'of',
 'genome',
 '68',
 'replication',
 'without',
 'the',
 'requirement',
 'for',
 'high',
 'containment',
 '(',
 '9',
 ',',
 '10',
 ')',
 '(',
 'figure',
 '1A',
 ')',
 '.']

In [46]:
solr_index_json_list = []
with open('/home/santosh/Work/Datasets/cord-19-json/data_dict_paragraphs.json', 'r') as infile:
    for line in infile:
        data = json.loads(line)
        solr_index_json_list.append(data)
        
with open('/home/santosh/Work/Datasets/cord-19-json/solr_paragraphs_index.json','w') as outfile:
    json.dump(solr_index_json_list,outfile)        

In [47]:
data

{'source': '10.1126/sciadv.aax4761',
 'title': 'More than efficacy revealed by single-cell analysis of antiviral therapeutics',
 'authors': 'Liu, Wu; Caglar, Mehmet U.; Mao, Zhangming; Woodman, Andrew; Arnold, Jamie J.; Wilke, Claus O.; Cameron, Craig E.',
 'index_key': 180881,
 'paragraph': 'Fig. S2. Evaluation of 2′-C-Me-A, a PV RdRp inhibitor. Fig. S3. Translation of eGFP mRNA and activation of fluorophore are normal in the presence of GA and GS. Fig. S4 Evaluation of GS, an HSP90 inhibitor. Fig. S5 Evaluation of an antiviral drug combination: 2′-C-Me-A and GS. Fig. S6 Evaluation of inhibitor-specific signatures.'}

In [32]:
from fse import IndexedList
paragraphs_index = IndexedList(paragraphs)

In [33]:
import pickle

def dump(data,filename):
    file = open(result_dir_path+filename+'.bin','wb')
    pickle.dump(data, file)
    file.close()

In [34]:
result_dir_path = '/home/santosh/Work/models/word2vec/CORD-19/'
dump(paragraphs_index, 'paragraphs_index_w2v_2017')

In [35]:
# SIF embeddings
from fse.models import uSIF
sif_model = uSIF(covid_trained_model, workers=4, lang_freq="en")

sif_model.train(paragraphs_index)

2020-04-04 19:39:44,891 : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en
2020-04-04 19:39:45,051 : INFO : scanning all indexed sentences and their word counts
2020-04-04 19:39:45,259 : INFO : finished scanning 180882 sentences with an average length of 142 and 25734978 total words
2020-04-04 19:39:45,308 : INFO : estimated memory for 180882 sentences with 200 dimensions and 241336 vocabulary: 323 MB (0 GB)
2020-04-04 19:39:45,309 : INFO : initializing sentence vectors for 180882 sentences
2020-04-04 19:39:45,876 : INFO : pre-computing uSIF weights for 241336 words
2020-04-04 19:39:46,452 : INFO : begin training
2020-04-04 19:39:51,490 : INFO : PROGRESS : finished 95.76% with 173215 sentences and 12197889 words, 34643 sentences/s
2020-04-04 19:39:51,690 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-04-04 19:39:51,691 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-04-04 19:39:51,691 : INFO : worker 

(180882, 12711460)

In [36]:
result_dir_path = '/home/santosh/Work/models/word2vec/CORD-19/'
sif_model.save(result_dir_path+'CORD-19-p2v_sif_2017.bin')

2020-04-04 19:40:00,839 : INFO : saving uSIF object under /home/santosh/Work/models/word2vec/CORD-19/CORD-19-p2v_sif_2017.bin, separately None
2020-04-04 19:40:00,840 : INFO : storing np array 'vectors' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-p2v_sif_2017.bin.wv.vectors.npy
2020-04-04 19:40:01,784 : INFO : storing np array 'vectors' to /home/santosh/Work/models/word2vec/CORD-19/CORD-19-p2v_sif_2017.bin.sv.vectors.npy
2020-04-04 19:40:06,810 : INFO : saved /home/santosh/Work/models/word2vec/CORD-19/CORD-19-p2v_sif_2017.bin


# Test Embeddings

In [ ]:
import pickle
from gensim.summarization import summarize

def load(filename):
    file = open(path_w2v+filename+'.bin','rb')
    data = pickle.load(file)
    file.close()
    return data

In [ ]:
path_w2v = '/home/santosh/Work/models/word2vec/CORD-19/'

paragraphs_index = load('paragraphs_index')



In [ ]:
from gensim.models import KeyedVectors
sif_model = KeyedVectors.load(path_w2v+'CORD-19-p2v_sif_2017.bin')

In [37]:
import re
def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
         "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    step7 = step6.replace("[ ", "[").replace(" ]", "]")
    return step7.strip()

def extract_query_result(sv_query_result):
    result_sentences =[]
    for each_result in sv_query_result:
        result_sentences.append(untokenize(each_result[0]))
     
    return result_sentences

In [51]:
query_result = sif_model.sv.similar_by_sentence(nltk.word_tokenize(query), model=sif_model, topn=5)
query_result

2020-04-04 21:01:34,670 : INFO : scanning all indexed sentences and their word counts
2020-04-04 21:01:34,671 : INFO : finished scanning 1 sentences with an average length of 7 and 7 total words
2020-04-04 21:01:34,673 : INFO : removing 5 principal components took 0s


[(57520, 0.7144672870635986),
 (112098, 0.7144672870635986),
 (13451, 0.6943836808204651),
 (28077, 0.692663311958313),
 (22543, 0.6777405142784119)]

In [50]:
list_indexs = []
for each_result in query_result:
    list_indexs.append(each_result[0])


In [43]:
# query = "pre-existing pulmonary disease SARS-Cov2 Hypertension" 

query = "What is the incubation days of SARS-CoV-2" 
# query = "incubation days coronavirus 2019-nCoV"#  COVID-19
# query = 'socio economic poverty behaviour'

# query = 'what is the influence of pre-existing diseases and comorbidities'
# query = 'public health mitigation measures that could be effective for control'
# query = 'socio-economic and behavioral factors to understand the economic impact of the SARS-CoV-2 virus and whether there were differences. '
# query = 'what are the risk factors for death in COVID-19'
# query = 'what is the basic reproductive number of SARS-CoV-2 in days'
# query = 'what is the serial interval days SARS-CoV-2'
# query = 'what do we know about the environmental factors influencing SARS-CoV-2'
# query = 'what do we know about drugs using to treat SARS-CoV-2'
# query = 'Transmission dynamics of the virus SARS-CoV-2'
# query ='risk of fatality among symptomatic hospitalized patients'
# query = 'Efforts targeted at a universal coronavirus vaccine'
# query = ''

query_result = sif_model.sv.similar_by_sentence(nltk.word_tokenize(query), model=sif_model, indexable=paragraphs_index.items, topn=10)

extract_query_result(query_result)
# summarize(' '.join(extract_query_result(query_result)), ratio =0.2, split=True)

2020-04-04 19:42:32,148 : INFO : scanning all indexed sentences and their word counts
2020-04-04 19:42:32,150 : INFO : finished scanning 1 sentences with an average length of 7 and 7 total words
2020-04-04 19:42:32,151 : INFO : removing 5 principal components took 0s


['Case 3 developed symptoms 4 days after her husband and 5 days after the couple had left Wuhan. The incubation period of SARS-CoV-2 is currently estimated at around 3-7 days [5, 10, 11]. Therefore, she may have acquired the infection from her husband, although this cannot be proved.',
 'Case 3 developed symptoms 4 days after her husband and 5 days after the couple had left Wuhan. The incubation period of SARS-CoV-2 is currently estimated at around 3-7 days [5, 10, 11]. Therefore, she may have acquired the infection from her husband, although this cannot be proved.',
 'Estimates for the incubation periods of SARS, MERS, and COVID-19.',
 '\uf06c Second, E → I. Relevant evidence shows that the longest incubation period for 2019-nCoV is 14 days and the shortest is 1 day. The Exposed status (E) may be transformed into an infected status (I) after the incubation period.',
 'Since the infection time of most patients could not be determined, we did not analyze the incubation period. It was re